### 환경 setup

In [ ]:
!apt-get install g++ openjdk-8-jdk python3-dev python3-pip curl

In [ ]:
!python3 -m pip install --upgrade pip

In [ ]:
!python3 -m pip install konlpy

In [4]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
import re
import pandas as pd

### 클롤링 데이터 load

In [5]:
df = pd.read_csv('/content/sample_data/naverblogs_crawling_selenium_Jul15-31.csv')
df.head(3)

,URL,Date,Like,Content,Hashtag
0,https://blog.naver.com/yesemi/222452707130,2021. 7. 31. 23:59,114.0,"2021년 7월 24일 토요일 < 오늘 배출된 플라스틱 > 어묵 2, 빼빼로 1, ...",#제로웨이스트 #제로웨이스터 #zerowaste #plastic #plasticdi...
1,https://blog.naver.com/ddoni89/222452706700,2021. 7. 31. 23:59,10.0,안녕하세요? 또니입니다! 오늘은 육아 중 실천할 수 있는 제로웨이스트 활동을 찾아볼...,#제로웨이스터 #제로웨이스트 #일회용품줄이기 #중고거래 #제로웨이스트육아
2,https://blog.naver.com/answn986123/222452705706,2021. 7. 31. 23:58,1.0,어느새 7월도 훌쩍_. 게 눈 감추듯 흘러갔다. 미루고 미루다 결국 단벌 감상은 ...,NaN


In [6]:
df.shape

(2827, 5)

### Content 컬럼이 missing value인 row 처리

In [ ]:
df.isnull().sum()

URL          0
Date        80
Like       150
Content    113
Hashtag    261
dtype: int64

In [22]:
dataframe = df.copy()

In [23]:
dataframe = dataframe.dropna(subset=['Content'])
dataframe.reset_index(drop=True, inplace=True)

In [9]:
dataframe.isnull().sum()

URL          0
Date         1
Like       159
Content      0
Hashtag    261
dtype: int64

In [42]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2698 entries, 0 to 2697
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   URL      2698 non-null   object 
 1   Date     2697 non-null   object 
 2   Like     2539 non-null   float64
 3   Content  2698 non-null   object 
 4   Hashtag  2437 non-null   object 
dtypes: float64(1), object(4)
memory usage: 105.5+ KB


In [25]:
dataframe.head()

,URL,Date,Like,Content,Hashtag
0,https://blog.naver.com/yesemi/222452707130,2021. 7. 31. 23:59,114.0,"2021년 7월 24일 토요일 < 오늘 배출된 플라스틱 > 어묵 2, 빼빼로 1, ...",#제로웨이스트 #제로웨이스터 #zerowaste #plastic #plasticdi...
1,https://blog.naver.com/ddoni89/222452706700,2021. 7. 31. 23:59,10.0,안녕하세요? 또니입니다! 오늘은 육아 중 실천할 수 있는 제로웨이스트 활동을 찾아볼...,#제로웨이스터 #제로웨이스트 #일회용품줄이기 #중고거래 #제로웨이스트육아
2,https://blog.naver.com/answn986123/222452705706,2021. 7. 31. 23:58,1.0,어느새 7월도 훌쩍_. 게 눈 감추듯 흘러갔다. 미루고 미루다 결국 단벌 감상은 ...,NaN
3,https://blog.naver.com/sms965/222452694793,2021. 7. 31. 23:47,3.0,#챌린저스 #올바른분리수거실천하기 #02요즘에는 아이스팩 안에 순수 물이 들어 있어...,#챌린저스 #올바른분리수거실천하기 #02 #아이스팩버리기 #페트병분리 #분리수거 #...
4,https://blog.naver.com/crazyelis/222452690732,2021. 7. 31. 23:43,0.0,코로나가 계속 지속되면서 도시락싸서 다니시는 분들 많을 것 같아요. (이젠 코로나 ...,#파룸체험단 #락앤락 #락앤락도시락 #도시락싸바싸바 #직장인도시락 #도시락 #다이어...


### 첫번째 content로 test해보기

In [11]:
text = dataframe['Content'][0]

In [12]:
text

'2021년 7월 24일 토요일 < 오늘 배출된 플라스틱 > 어묵 2, 빼빼로 1, 사과즙 2, 유산균 1, 인공누액 2.모닝 사과즙. 아이가 좋아하는 어묵으로 반찬도 하고 국도 끓이고. 마트에서 그냥 빠져나오지 못하고 집어든 빼빼로 한 봉지. 그리고 유산균과 인공누액 두개. 소소하고 평범한 하루에 배출된 소소하지 않은 쓰레기들. 오늘 플라스틱 일기, 끝.#제로웨이스트 #제로웨이스터 #zerowaste #plastic #plasticdiary #eco #earthday #플라스틱일기 #플라스틱일기챌린지 #챌린지 #쓰레기일기 #매일기록하기 #나먼저변화하기 #오늘부터 #함께해요 #어스데이 #지구의날 #환경 #기후 #일기 #쓰레기 #줄이기 #실천하기 #분리배출 #분리수거 #플라스틱 #비닐 #소소한하루 #소소하지않은쓰레기들'

In [13]:
def apply_regular_expression(text):
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
    result = hangul.sub('', text)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
    return result

In [14]:
re_result = apply_regular_expression(text)

In [15]:
re_result

'년 월 일 토요일  오늘 배출된 플라스틱  어묵  빼빼로  사과즙  유산균  인공누액 모닝 사과즙 아이가 좋아하는 어묵으로 반찬도 하고 국도 끓이고 마트에서 그냥 빠져나오지 못하고 집어든 빼빼로 한 봉지 그리고 유산균과 인공누액 두개 소소하고 평범한 하루에 배출된 소소하지 않은 쓰레기들 오늘 플라스틱 일기 끝제로웨이스트 제로웨이스터      플라스틱일기 플라스틱일기챌린지 챌린지 쓰레기일기 매일기록하기 나먼저변화하기 오늘부터 함께해요 어스데이 지구의날 환경 기후 일기 쓰레기 줄이기 실천하기 분리배출 분리수거 플라스틱 비닐 소소한하루 소소하지않은쓰레기들'

In [16]:
from konlpy.tag import Okt
from collections import Counter

In [20]:
okt = Okt()
nouns = okt.nouns(re_result)

### 광고성글 제외시키기

In [19]:
exclude= ['원고료', '광고', '홍보', '협찬','협찬받아', '무상', '지원받아', '제공받아','제공받고','당첨','이벤트','당첨 후']

In [43]:
dfcopy = dataframe.copy()

for idx in dfcopy.index:
    hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')
    result = hangul.sub('', str(dfcopy.loc[idx,'Content']))
    words_row = result.split(" ")
    for x in words_row:
        if x in exclude: 
            dfcopy = dfcopy.drop([idx])
            break  

In [44]:
dfcopy.info() # 광고성 단어 포함된 글 삭제 후, contents개수 2698 --> 2445로 감소

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2445 entries, 0 to 2697
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   URL      2445 non-null   object 
 1   Date     2444 non-null   object 
 2   Like     2301 non-null   float64
 3   Content  2445 non-null   object 
 4   Hashtag  2202 non-null   object 
dtypes: float64(1), object(4)
memory usage: 194.6+ KB


### 말뭉치 생성 및 명사 추출

In [45]:
# 말뭉치 생성

#corpus = "".join(dfcopy['Content'].tolist())

dfcopy['Content'] = dfcopy['Content'].astype(str)

corpus=""
for idx in dfcopy.index:
    corpus += (" " + str(dfcopy.loc[idx, 'Content']))

In [46]:
re_corpus = apply_regular_expression(corpus)

In [66]:
noun_corpus = okt.nouns(re_corpus)

In [67]:
len(noun_corpus) # 총 명사갯수: 613,334개

613334

In [68]:
noun_corpus_unique = set(noun_corpus)
len(noun_corpus_unique) # 중복없이, noun_corpus_unique는 26,499 종류의 명사 존재함.

26499

### stopword제거 없이 빈도수 확인시 발생하는 문제 확인

In [41]:
# stopword 제거 없이 빈도수 보면 다음과 같이 불필요한 단어가 높은 순위에 있음
freq = pd.Series(noun_corpus).value_counts()
freq[:10]

것      7009
수      6765
사용     6444
제로     5587
웨      5237
이스트    5166
제품     3403
때      3327
환경     3206
생각     3190
dtype: int64

In [ ]:
counter = Counter(noun_corpus)
counter.most_common(10)

[('것', 6634),
 ('수', 6387),
 ('사용', 6040),
 ('제로', 4981),
 ('웨', 4657),
 ('이스트', 4596),
 ('제품', 3303),
 ('때', 3100),
 ('환경', 3098),
 ('생각', 2943)]

In [ ]:
#다음과 같이 추가적으로 stopwords를 더해서 '제로 웨이스트'와 같은 단어들은 제외되어야한다.
available_counter = Counter({x:counter[x] for x in counter if len(x)>1})
available_counter.most_common(10) 

[('사용', 6040),
 ('제로', 4981),
 ('이스트', 4596),
 ('제품', 3303),
 ('환경', 3098),
 ('생각', 2943),
 ('플라스틱', 2743),
 ('비누', 2645),
 ('쓰레기', 2236),
 ('친환경', 2105)]

### stopwords 처리로, 문제 해결

In [59]:
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
stopwords[:10]

[['휴'],
 ['아이구'],
 ['아이쿠'],
 ['아이고'],
 ['어'],
 ['나'],
 ['우리'],
 ['저희'],
 ['따라'],
 ['의해']]

TODO: stopwords에서 '우리', '저희', '함께' 는 빼야하는것같다. 검색 주제의 공동체 성격을 유지하기위해서.

In [60]:
type(stopwords)

list

In [62]:
stopwords.remove(['우리'])

In [63]:
stopwords.remove(['저희'])

In [ ]:
stopwords.remove(['함께'])

In [71]:
stopwords[:10]

[['휴'], ['아이구'], ['아이쿠'], ['아이고'], ['어'], ['따라'], ['의해'], ['을'], ['를'], ['에']]

In [72]:
zerowaste_stopwords = ['제로','웨','이스트','이스터','제로 웨이스트','제로웨이스트',\
                       '제로 웨이스터','제로웨이스터']

In [73]:
for word in zerowaste_stopwords: stopwords.append(word)

In [74]:
# 한글자 키워드 제거
noun_corpus = [x for x in noun_corpus if len(x)>1]

In [75]:
# 불용어 제거
noun_corpus = [x for x in noun_corpus if x not in stopwords]

In [76]:
freq = pd.Series(noun_corpus).value_counts()
freq[:10]

사용      6364
제품      3354
환경      3164
생각      3155
플라스틱    3062
비누      2926
쓰레기     2503
친환경     2154
실천      2010
우리      1945
dtype: int64